In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import Point, LineString, MultiLineString, Polygon, MultiPolygon
from tqdm.notebook import tqdm
import time
from datetime import datetime

import momepy as mm
import networkx as nx

import math

from contextlib import redirect_stdout

import io
import sys

from shapely.ops import nearest_points
from shapely.ops import unary_union

In [ ]:
# проверка остановки в буффере - если даже по вохдуху не попадает - не искать

In [2]:
graph = gpd.read_file('../lines_to_graph/with_ped/new_graph_with_ped.json',
                        encoding='utf-8')

In [3]:
nodes = gpd.read_file('../lines_to_graph/with_ped/all_nodes_with_ped.json',
                        encoding='utf-8')

In [10]:
buildings = gpd.read_file('./all_bui.json',encoding='utf-8')

In [4]:
stps_big = pd.read_csv('../lines_to_graph/total/df_stop_seq_all_n_samobl.csv',
                        encoding='utf-8', sep=';')

In [8]:
stps_big = stps_big[['stop_id', 'stop_name', 'lat', 'lon']]
stps_big = stps_big.drop_duplicates(subset=['stop_id']).reset_index(drop=True)

In [ ]:
lst_geo=[]
lst_lat=list(stps_big.lat)
lst_lon=list(stps_big.lon)
i=0
for i in tqdm(range(len(stps_big))):
    one_pt = Point(lst_lon[i], lst_lat[i])
    lst_geo.append(one_pt)
# 

In [ ]:
gdf_stops = gpd.GeoDataFrame(stps_big, geometry=lst_geo)
gdf_stops.crs='epsg:4326'

In [34]:
# убрать жд пути из графа
graph_ped = graph[graph.type_ped != 'no_ped'].reset_index(drop=True)[['link_id', 'geometry']]

In [13]:
shrt_bui = buildings[['osm_id', 'geometry']].copy()

In [ ]:
def intersect_using_spatial_index(source_gdf, intersecting_gdf):
    """
    Conduct spatial intersection using spatial index for candidates GeoDataFrame to make queries faster.
    Note, with this function, you can have multiple Polygons in the 'intersecting_gdf' and it will return all the points
    intersect with ANY of those geometries.
    """
    source_sindex = source_gdf.sindex
    possible_matches_index = []

    # 'itertuples()' function is a faster version of 'iterrows()'
    for other in intersecting_gdf.itertuples():
        bounds = other.geometry.bounds
        c = list(source_sindex.intersection(bounds))
        possible_matches_index += c

    # Get unique candidates
    unique_candidate_matches = list(set(possible_matches_index))
    possible_matches = source_gdf.iloc[unique_candidate_matches]

    # Conduct the actual intersect
    result = possible_matches.loc[possible_matches.intersects(intersecting_gdf.unary_union)]
    return result
########

In [ ]:
def find_closest_stop(gdf_one_bui, gdf_stops, gdf_buff):
    uu_stops = gdf_stops.geometry.unary_union
    centr_bui = gdf_one_bui.geometry[0].centroid
    nrst_pt = nearest_points(centr_bui, uu_stops)[1]
    
    nearest_stop_id = list(gdf_stops[gdf_stops.geometry == nrst_pt]['stop_id'])[0]
    nearest_stop_geo = list(gdf_stops[gdf_stops.geometry == nrst_pt]['geometry'])[0]
    if gdf_buff.geometry[0].intersects(nearest_stop_geo):
        return [nearest_stop_id, nearest_stop_geo]
    else:
        return []

In [183]:
def get_closest_node(gdf_obj, linkid, graph_in_buff):
    line_geo = list(graph_in_buff[graph_in_buff.link_id == linkid]['geometry'])[0]
    start = Point(line_geo.coords[0])
    end = Point(line_geo.coords[-1])
    obj_centr = gdf_obj['geometry'][0]#.centroid

    if start.distance(obj_centr) < end.distance(obj_centr):
        return start
    else:
        return end
# 

In [206]:
def get_shrt_pth(gdf_one_bui, gdf_buff, gdf_stops, graph_ped):

    try:
        nearest_stop = find_closest_stop(gdf_one_bui, gdf_stops, gdf_buff)
    except:
        nearest_stop = []
    #
    if nearest_stop != []:
        gdf_near_stop = gpd.GeoDataFrame(data=[nearest_stop], columns=['stop_id', 'geometry'])
        gdf_near_stop.crs=graph_ped.crs
        graph_in_buff = intersect_using_spatial_index(graph_ped, gdf_buff[['geometry']])

        text_trap = io.StringIO()
        with redirect_stdout(text_trap):
            closest_link_bui = mm.get_network_id(gdf_bui, graph_in_buff, 'link_id')[0]
            closest_link_stop = mm.get_network_id(gdf_near_stop, graph_in_buff, 'link_id')[0]

        closest_node_bui = get_closest_node(gdf_bui, closest_link_bui, graph_in_buff)
        closest_node_stop = get_closest_node(gdf_near_stop, closest_link_stop, graph_in_buff)

        one_G = mm.gdf_to_nx(graph_in_buff)
        res_lst = nx.shortest_path(one_G, closest_node_bui.coords[0], closest_node_stop.coords[0])

        if len(res_lst) == 1:
            line_path = LineString([closest_node_bui, closest_node_stop])
            path_lngth = 0
        elif len(res_lst) == 0:
            line_path = LineString([closest_node_bui, closest_node_stop])
            path_lngth = -1
        else:
            line_path = LineString(res_lst)
            path_lngth = 1
    # 
        return [nearest_stop[0], path_lngth, res_lst, line_path]
    else:
        return []

In [171]:
# list(shrt_bui.osm_id).index(40707382)

4615

In [15]:
np_shbu = shrt_bui.to_numpy()
ind_oi_bui = list(shrt_bui.columns).index('osm_id')
ind_geo_bui = list(shrt_bui.columns).index('geometry')

In [207]:
lst_total_path = []
i=0
# for i in tqdm(range(len(np_shbu))):
for i in tqdm(range(2)):
    one_bui_oi = np_shbu[i, ind_oi_bui]
    one_bui_geo = np_shbu[i, ind_geo_bui]
    
    gdf_one_bui = gpd.GeoDataFrame(geometry=[one_bui_geo])
    gdf_one_bui.crs = 'epsg:4326'

    gdf_buff = gdf_one_bui.copy()
    gdf_buff.geometry = gdf_buff.geometry.to_crs('epsg:32637').buffer(1000).to_crs('epsg:4326')
    
    lst_path = get_shrt_pth(gdf_one_bui, gdf_buff, gdf_stops, graph_ped)
    
    if lst_path != []:
        lst_total_path.append([one_bui_oi] + lst_path)
    #
# 

C:\Users\ochristm\anaconda3\lib\site-packages\momepy\elements.py:696: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  buildings_c["geometry"] = buildings_c.centroid  # make centroids

Snapping: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 16.95it/s]

Snapping: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 18.52it/s]
C:\Users\ochristm\anaconda3\lib\site-packages\momepy\utils.py:202: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_network[length] = gdf_network.geometry.length
C:\Users\ochristm\anaconda3\lib\site-packages\momepy\elements.py:696: UserWarning: Geometry is in a geographic CRS. Results from 'cent

In [210]:
gdf_path = gpd.GeoDataFrame(lst_total_path, 
                            columns = ['osm_id_bui','stop_id_ya',
                                       'is_valid', 'node_seq', 'geometry'])
gdf_path.crs = graph_ped.crs